In [1]:
!pip install langchain
!pip install torch
!pip install transformers
!pip install sentence-transformers
!pip install datasets
!pip install faiss-cpu
!pip install -U langchain-community
!pip install tqdm
!pip install bitsandbytes
!pip install huggingface_hub
!pip install --upgrade accelerate
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from langchain_community.document_loaders import HuggingFaceDatasetLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from datasets import load_dataset
from tqdm import tqdm
import multiprocessing as mp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 13.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.5/150.5 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.4 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 r

NameError: name '_name_' is not defined

In [ ]:
def load_and_process_dataset(args):
    if len(args) == 3:
        dataset_name, input_col, output_col = args
        config = None
    else:
        dataset_name, input_col, output_col, config = args
    
    if config:
        dataset_name = f"{dataset_name}/{config}"
    
    loader = HuggingFaceDatasetLoader(dataset_name, output_col)
    data = loader.load()
    
    dataset = load_dataset(dataset_name)
    processed_data = []
    for i, doc in tqdm(enumerate(data), desc=f"Processing {dataset_name}", leave=False):
        input_text = dataset['train'][i][input_col]
        doc.metadata['full_qa'] = f"Input: {input_text}\nOutput: {doc.page_content}"
        processed_data.append(doc)
    
    return processed_data

def process_in_batches(docs, batch_size=1000):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]

if __name__ == "__main__":
    datasets = [
        ("ruslanmv/ai-medical-chatbot", "Patient", "Doctor"),
        ("lavita/ChatDoctor-iCliniq", "input", "answer_chatdoctor"),
        ("xDAN-datasets/medical_meadow_wikidoc_patient_information_6k", "input", "output"),
        ("mlabonne/medical_meadow_medqa", "instruction","output"),
        ("medalpaca/medical_meadow_medical_flashcards", "input", "output"),
        ("lavita/ChatDoctor-HealthCareMagic-100k","input","output")
    ]

    # Parallel processing of datasets
    print("Loading and processing datasets:")
    with mp.Pool(processes=mp.cpu_count()) as pool:
        all_data = list(tqdm(pool.imap(load_and_process_dataset, datasets), total=len(datasets), desc="Datasets"))
    all_data = [item for sublist in all_data for item in sublist]

    print(f"Total documents loaded: {len(all_data)}")

    print("Splitting documents:")
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = []
    for doc in tqdm(all_data, desc="Splitting"):
        docs.extend(text_splitter.split_documents([doc]))

    print(f"Total chunks after splitting: {len(docs)}")

    embeddings = HuggingFaceEmbeddings()

    print("Creating FAISS index...")
    db = None
    batch_size = 1000
    total_batches = len(docs) // batch_size + (1 if len(docs) % batch_size != 0 else 0)

    for i, batch in enumerate(tqdm(process_in_batches(docs, batch_size), total=total_batches, desc="Vectorizing")):
        if i == 0:
            db = FAISS.from_documents(batch, embeddings)
        else:
            db.add_documents(batch)

    print("Saving FAISS index...")
    db.save_local("RAG_DB_Multiple_Answer_Based")
    print("FAISS index created and saved.")

    print("Loading FAISS index...")
    loaded_db = FAISS.load_local("RAG_DB_Multiple_Answer_Based", embeddings, allow_dangerous_deserialization=True)
    print("FAISS index loaded.")

    def query_database(query, k=6):
        results = loaded_db.similarity_search(query, k=k)
        for doc in results:
            print("Retrieved answer:", doc.page_content[:100] + "...")
            print("Full Q&A:")
            print(doc.metadata['full_qa'])
            print("\n---\n")

    query_database("What are the symptoms of diabetes?")

Loading and processing datasets:


Datasets:   0%|          | 0/6 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2566: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 a

Generating train split:   0%|          | 0/5850 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/10178 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/256916 [00:00<?, ? examples/s]

...

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Creating FAISS index...


Vectorizing:  10%|▉         | 45/457 [3:24:34<34:44:42, 303.60s/it]

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session